In [25]:
def mse(a1, a2):
    se = np.square(np.subtract(a1,a2))
    mse = se.mean()
    return mse

In [26]:
print("Overall MSE: ", mse(gt,pr))
ch =['tas', 'psl', 'pr']
for i in range(3):
    print(ch[i],": mse = ",mse(gt[:,:,i], pr[:,:,i]))

Overall MSE:  0.00388691174544378
tas : mse =  0.0036669841558973322
psl : mse =  0.004464195120315682
pr : mse =  0.0035295559601183266


In [28]:
import numpy as np
from interpolate import *
import cartopy.crs as ccrs
import xarray as xr

path = "/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/data/"
inn = xr.open_dataset(path+"compress.isosph.CESM2-FV2.historical.r1i1p1f1.Output.nc")
inn['time'][:5]

<xarray.DataArray 'time' (time: 5)>
array([cftime.DatetimeNoLeap(1850, 1, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 14, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 4, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 5, 15, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 1850-05-15 12:00:00
Attributes:
    standard_name:  time
    axis:           T

In [36]:
#fname="./data/Exp7_CESM2_r1i1p1f1_historical_Input.nc"
#from data_loader import *
#lon, lat, _ =load_ncdf_to_SphereIcosahedral(fname, 6)
lon = np.load("./data/lon_level6.npy")
lat = np.load("./data/lat_level6.npy")
print(np.shape(lon), np.shape(lat))

(40962,) (40962,)


In [37]:
def visualize_2d_all(gt, pr, longitude, latitude, title, export_path=None):
    """Visualize the data on a 2D map

    Args:
        x (numpy.array): numpy array with data the size of the longitude and latitude
        longitude (numpy.array): longitude coordinates
        latitude (numpy.array): latitude coordinates
        export_path (string): path and name for saving
    """

    fig = plt.figure(figsize=(70, 10))
    ax = fig.add_subplot(1, 3, 1, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()

    plt.subplot(1, 3, 1)
    plt.scatter(longitude, latitude, s=20, c=gt, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.clim(0, 1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.title("ground truth "+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)

    ax = fig.add_subplot(1, 3, 2, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    plt.subplot(1, 3, 2)
    plt.scatter(longitude, latitude, s=20, c=pr, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.clim(0, 1)
    plt.title("prediction "+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)
    
    ax = fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    plt.subplot(1, 3, 3)
    plt.scatter(longitude, latitude, s=20, c=gt-pr, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    plt.xticks([-180,-90, 0,+90, 180])
    plt.yticks([-90,-45,0,45,90])
    plt.clim(-1, 1)
    plt.title("error (gt-pr)"+title)
    plt.colorbar(cmap=plt.get_cmap("RdYlBu_r"),fraction=0.046, pad=0.04)
    if export_path:
        plt.savefig(export_path)
        plt.clf()
        plt.cla()
        plt.close()

    else:
        plt.show()

In [47]:
import os
ch =['tas', 'psl', 'pr']

PATH = ["/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_1/", "/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_2/", "/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_3/"]
fname_gt = ["groundtruth_20_tensor(0.0039).npy", "groundtruth_20_tensor(0.0067).npy", "groundtruth_20_tensor(0.0079).npy"]
fname_pr = ["prediction_20_tensor(0.0039).npy", "prediction_20_tensor(0.0067).npy", "prediction_20_tensor(0.0079).npy"]
for k in range(len(PATH)):
    os.mkdir(PATH[k]+"/fig/")
    gt = np.load(PATH[k]+fname_gt[k])
    pr = np.load(PATH[k]+fname_pr[k])
    gt = np.swapaxes(gt[:,0,:,:],1,2)
    pr = np.swapaxes(pr[:,0,:,:],1,2)
    print(PATH[k], fname_gt[k], fname_pr[k], np.shape(gt), np.shape(pr)) 
    print("Overall MSE: ", mse(gt,pr))
    for l in range(3):
        print(ch[l],": mse = ",mse(gt[:,:,l], pr[:,:,l]))
    for i in range(len(gt)):
        for j in range(3):
            visualize_2d_all(gt[i,:,j],pr[i,:,j], lon, lat, ch[j]+" "+str(i), export_path=PATH[k]+"/fig/"+ch[j]+"_testfile_"+str(i)+".png")


/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_1/ groundtruth_20_tensor(0.0039).npy prediction_20_tensor(0.0039).npy (198, 40962, 3) (198, 40962, 3)
Overall MSE:  0.00388691174544378
tas : mse =  0.0036669841558973322
psl : mse =  0.004464195120315682
pr : mse =  0.0035295559601183266
/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_2/ groundtruth_20_tensor(0.0067).npy prediction_20_tensor(0.0067).npy (198, 40962, 3) (198, 40962, 3)
Overall MSE:  0.009744112743092257
tas : mse =  0.00801022522517972
psl : mse =  0.011021569525543454
pr : mse =  0.01020054347855362
/Users/sookim/Desktop/aibedo_sunet/aibedo/skeleton_framework/saved_model_convlstmunet_3/ groundtruth_20_tensor(0.0079).npy prediction_20_tensor(0.0079).npy (198, 40962, 3) (198, 40962, 3)
Overall MSE:  0.009945296092965187
tas : mse =  0.00861344944290944
psl : mse =  0.010843084393764654
pr : mse =  0.010379354442221439
